## Gemini Pro Vision Video 분석

Feedback : shins777@gmail.com. 

이 Colab 은 다양한 비즈니스 요구사항에 적용할수 있는 비디오 분석에 대한 예제입니다.
Gemini Pro vision을 활용해서 비디오를 분석합니다.

* 코드는 Gemini Pro API 기반으로 처리하며 그에 따른 API는 아래 링크 참고하세요.  
    * 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

##### 참고: 이곳에서 사용되는 비디오 파일은 교육목적으로 무료 웹 상에서 다운로드 받은 이미지입니다. 상업적인 용도로 사용하지 말아주세요. 혹시 저작권의 문제가 있다면 위의 이메일로 연락 부탁드립니다.

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


### GCP 사용자 인증 / 환경설정

GCP 인증방법은 아래와 URL 정보를 참고하여 GCP에 접근 하는 환경을 구성해야 합니다. 
* https://cloud.google.com/docs/authentication?hl=ko
* 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [2]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
# from google.colab import auth
# auth.authenticate_user()

### GCP 프로젝트 및 리전 설정
본인의 GCP 환경에 맞게 아래 설정을 구성하세요.  
* 구글의 최신버전인 gemini pro 사용을 권고드립니다.   
* 만일, 기본 버전 text bison 을 사용하려한다면, 참조하는 class 가 다르므로 주의하세요.  
* 현재 Gemini는 한국리전(asia-northeast3)을 통해서 접근이 가능합니다.

In [3]:
model_name="gemini-pro-vision"
project="ai-hangsik"
location="asia-northeast3"

#### Google Generative AI를 사용하기 위한 aiplatform initialize.

In [4]:
from google.cloud import aiplatform
aiplatform.init(project=project, location = location)

### Gemini Pro vision - Multimodal
비디오 base64 인코딩을 위한 함수 

In [ ]:
import base64
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

def get_obj_content(coding='utf-8', obj_loc=""):
    with open(obj_loc, 'rb') as f:
        obj_data = base64.b64encode(f.read()).decode(coding)
        return obj_data


### Video 분석

* 사용가능한 비디오 MIME type은 아래 정보 참고하세요.
    * https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts?hl=ko#video-requirements

* 참고 : 아래 예제에서 사용된 비디오는 아래 무료사이트에서 제공하는 비디오입니다. 교육의 목적으로 제공하고 있으니 상업적으로 사용하지 마시기 바랍니다.  
    * https://pixabay.com/videos/work-office-people-working-87593/

#### video 분석 1

In [8]:
def generate_video(obj):
  model = GenerativeModel("gemini-pro-vision")
  responses = model.generate_content(
    [obj, """비디오의 내용을 전체적으로 요약해주고 아래 내용을 구체적으로 설명해주세요.
    1. 사람들이 있는 장소를 알려주고, 무엇이 있나요?
    2. 노란색옷을 입은 사람은 주로 어떤 손을 사용하나요 ?
    3. 노트북은 어떤 브랜드인가요 ?
    4. 화면에 나타난 여자의 행동은 어떤까요 ?
    4. 검정옷을 입은 사람은 손에 무엇을 들고 있나요 ?
    """],
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.1,
        "top_p": 1,
        "top_k": 32
    },
    safety_settings=[],
    stream=True,
    )

  for response in responses:
      print(response.text, end="")

video_file_path = "../../../contents/videos/work(720p).mp4"

video_data = get_obj_content(obj_loc=video_file_path)
video1 = Part.from_data(data=base64.b64decode(video_data), mime_type="video/mp4")

generate_video(video1)

 1. 장소는 사무실이고, 세 명이 있습니다.
2. 오른손을 주로 사용합니다.
3. 노트북은 애플입니다.
4. 여자는 웃고 있습니다.
5. 검정옷을 입은 사람은 서류를 들고 있습니다.

####  video 분석 2

아래 예제에서 사용된 비디오는 아래 무료사이트에서 제공하는 비디오입니다. 교육의 목적으로 제공하고 있으니 상업적으로 사용하지 마시기 바랍니다.  
https://pixabay.com/videos/animate-teacher-class-student-22182/

In [9]:
def generate_video(obj):
  model = GenerativeModel("gemini-pro-vision")
  responses = model.generate_content(
    [obj, """전체적으로 비디오에 나타나는 상황을 자세히 설명해주세요.
    1. 주로 무엇을 그리고 있나요 ? 그리는 도구는 무엇인가요 ?
    2. 선생님의 머리길이는 어떠한가요 ?
    3. 학생은 몇명이고, 어느쪽 학생을 먼저 그리나요 ?
    4. 오른쪽의 학생은 무엇을 하고 있나요 ?
    5. 교실안에 지구본이 있나요? 어디위에 있나요 ?

    """],
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.4,
        "top_p": 1,
        "top_k": 32
    },
    safety_settings=[],
    stream=True,
    )

  for response in responses:
      print(response.text, end="")


video_file_path = "../../../contents/videos/classroom(720p).mp4"


video_data = get_obj_content(obj_loc=video_file_path)
video1 = Part.from_data(data=base64.b64decode(video_data), mime_type="video/mp4")

generate_video(video1)

 1. 초등학교 교실을 그리는 것 같습니다. 도구는 분필입니다.
2. 선생님의 머리길이는 짧습니다.
3. 학생은 3명이고, 오른쪽 학생을 먼저 그립니다.
4. 오른쪽 학생은 책을 읽고 있습니다.
5. 교실안에 지구본이 있습니다. 좌측 선생님 책상 위에 있습니다.

####  video 분석 3

아래 예제에서 사용된 비디오는 아래 무료사이트에서 제공하는 비디오입니다. 교육의 목적으로 제공하고 있으니 상업적으로 사용하지 마시기 바랍니다.  


In [10]:
def generate_video(obj):
  model = GenerativeModel("gemini-pro-vision")


  responses = model.generate_content(
    [obj, """전체적으로 비디오에 나타나는 상황을 자세히 설명해주세요.
    1. 칠판에 적힌 글씨는 그대로 적어주세요.
    2. 어떤 내용의 정보를 적고 있나요 ?

    """],

    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.4,
        "top_p": 1,
        "top_k": 32
    },
    safety_settings=[],
    stream=True,
    )

  for response in responses:
      print(response.text, end="")

video_file_path = "../../../contents/videos/text(1080p).mp4"

video_data = get_obj_content(obj_loc=video_file_path)
video1 = Part.from_data(data=base64.b64decode(video_data), mime_type="video/mp4")

generate_video(video1)

 1. 칠판에 적힌 글씨는 다음과 같습니다.

브랜드 전략
내부 브랜드
외부 브랜드

목적: Bring everyday
비전: 
미션: 
가치: 
퍼스낼리티: 
브랜드 목표: 


2. 브랜드 전략에 대한 정보를 적고 있습니다.